<a href="https://colab.research.google.com/github/yukinaga/minnano_kaggle/blob/main/section_3/02_hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ハイパーパラメータの調整
精度向上のために、ハイパーパラメータを調整します。 

## Optunaのインストール
ハイパーパラメータの最適化に使用するライブラリ、Optunaをインストールします。

In [ ]:
!pip install optuna

## データの準備
必要なライブラリの導入、データの読み込みと加工を行います。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import lightgbm as lgb
import optuna

train_data = pd.read_csv("train.csv")  # 訓練データ
test_data = pd.read_csv("test.csv") # テストデータ

test_id = test_data["PassengerId"]  # 結果の提出時に使用

data = pd.concat([train_data, test_data], sort=False)  # テストデータ、訓練データを結合

# カテゴリデータの変換
data["Sex"].replace(["male", "female"], [0, 1], inplace=True)
data["Embarked"].fillna(("S"), inplace=True)
data["Embarked"] = data["Embarked"].map({"S": 0, "C": 1, "Q": 2})

# 欠損値を埋める
data["Fare"].fillna(data["Fare"].mean(), inplace=True)
data["Age"].fillna(data["Age"].mean(), inplace=True)

# 新しい特徴量の作成
data["Family"] = data["Parch"] + data["SibSp"]

# 不要な特徴量の削除
data.drop(["Name", "PassengerId", "SibSp", "Parch", "Ticket", "Cabin"],
          axis=1, inplace=True)

# 入力と正解の作成
train_data = data[:len(train_data)]
test_data = data[len(train_data):]
t = train_data["Survived"]  # 正解
x_train = train_data.drop("Survived", axis=1)  # 訓練時の入力
x_test = test_data.drop("Survived", axis=1)  # テスト時の入力

x_train.head()

## データの分割
訓練用データを訓練用と検証用に分割します。

In [ ]:
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t, test_size=0.3, stratify=t)

## ハイパーパラメータの最適化
ハイパーパラメータ最適化のための関数を用意します。  
最適化には、Optunaというライブラリを使用します。  
https://github.com/optuna/optuna  
  
また、機械学習のアルゴリズムには決定木をベースにした「LightGBM」を使います。  
LightGBMは「勾配ブースティング」の一種で使い勝手が良く、多くのKaggle上位者に使用された実績があります。  
大量の決定木を使用し、ある決定木の予測結果から誤差の大きなデータをうまく予測できるように次の決定木を作成します。  
https://lightgbm.readthedocs.io/en/latest/


In [ ]:
categorical_features = ["Embarked", "Pclass", "Sex"]

def objective(trial):
    # ハイパーパラメータの探索範囲
    params = {
        "objective": "binary",  # 二値分類
        "max_bin": trial.suggest_int("max_bin", 200, 500),  # 特徴量の最大分割数
        "learning_rate": 0.05,  # 学習率
        "num_leaves": trial.suggest_int("num_leaves", 16, 128)  # 分岐の末端の最大数
    }

    # データセットの作成
    lgb_train = lgb.Dataset(x_train, t_train, categorical_feature=categorical_features)
    lgb_val = lgb.Dataset(x_valid, t_valid, reference=lgb_train, categorical_feature=categorical_features)

    # モデルの訓練
    model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_val],
                      verbose_eval=20,  # 学習過程の表示間隔
                      num_boost_round=500,  # 学習回数の最大値
                      early_stopping_rounds=10)  # 連続して10回性能が向上しなければ終了

    y_valid = model.predict(x_valid, num_iteration=model.best_iteration)  # 訓練済みのモデルを使用
    score = log_loss(t_valid, y_valid)  # 二値の交差エントロピー誤差
    return score

Optunaを使い、ハイパーパラメータを最適化します。


In [ ]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler())
study.optimize(objective, n_trials=30)

ベストなハイパーパラメータを表示します。

In [ ]:
print(study.best_params)

ベストなハイパーパラメータを使って予測を行います。

In [ ]:
# ベストなハイパーパラメータの設定
params = {
    "objective": "binary",  # 二値分類
    "max_bin": study.best_params["max_bin"],  # 特徴量の最大分割数
    "learning_rate": 0.05,  # 学習率
    "num_leaves": study.best_params["num_leaves"]  # 分岐の末端の最大数
}

# データセットの作成
lgb_train = lgb.Dataset(x_train, t_train, categorical_feature=categorical_features)
lgb_val = lgb.Dataset(x_valid, t_valid, reference=lgb_train, categorical_feature=categorical_features)

# モデルの訓練
model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_val],
                    verbose_eval=20,  # 学習過程の表示間隔
                    num_boost_round=500,  # 学習回数の最大値
                    early_stopping_rounds=10)  # 連続して10回性能が向上しなければ終了

y_test = model.predict(x_test, num_iteration=model.best_iteration)  # 訓練済みのモデルを使用

## 提出用のデータ
提出量データの形式を整え、CSVファイルに保存します。

In [ ]:
# 結果を0か1に
y_test = (y_test > 0.5).astype(int)

# 形式を整える
survived_test = pd.Series(y_test, name="Survived")
subm_data = pd.concat([test_id, survived_test], axis=1)

# 提出用のcsvファイルを保存
subm_data.to_csv("submission_titanic_hp.csv", index=False)

subm_data